In [2]:
from pymongo import MongoClient
from datetime import datetime
import numpy as np
from scipy import linalg
#client = MongoClient('mongodb://server.mcgarity.info:27017')
client = MongoClient()  # use local database by default
#db = client["borg_rapid_tests"]
db = client["borg_wingo_regress"]
collectionNames = sorted(db.collection_names())
collectionNames

[u'y16m05d20_100_1', u'y16m05d20_100_2']

In [3]:
list1 = [1,2]
list2 = [3,4]
A = np.array(list1)
A = np.vstack((A,list2))
B = linalg.inv(A)

C = np.dot(A,B)
C

array([[  1.00000000e+00,   0.00000000e+00],
       [  8.88178420e-16,   1.00000000e+00]])

In [8]:
nvars = 14 # 2 LIDs for each of 7 subcatchments
runs = db['y16m05d20_100_2']
cursor = runs.find()
ndocs = runs.count()
nruns = ndocs-1  # actual number of runs
start = 0
runDict = cursor[start]
X = runDict['variables']
X.insert(0,1.0)
Y = [runDict['volReduction']]
#X = np.vstack((X,runDict['variables']))
for runNum in range(start+1,ndocs-1):
    runDict = cursor[runNum]
    newRow = runDict['variables']
    newRow.insert(0,1.0)
    X = np.vstack((X,newRow))
    Y.append(runDict['volReduction'])
X = np.array(X)
Y = np.array(Y)
XT = X.transpose()
A = np.dot(XT,X)
Ainv = np.linalg.inv(A)
C = np.dot(XT,Y)
Theta = np.dot(Ainv,C)
Yhat = np.dot(X,Theta)
#print Y
#print Yhat
e = Y - Yhat
SSe = np.dot(e.transpose(),e)
Var_e = SSe/(nruns-nvars+1)
Cov = SSe * Ainv
Ybar = np.mean(Y)
Syy = np.dot(Y,Y) - nruns*Ybar*Ybar
Rsquare = 1.0 - SSe/Syy
Rsquare

0.9996743314465153